### 工作進度  

* 重新建立「量化技術分析」資料庫。 

In [1]:
import os
import sys
import pandas as pd
import datetime
import sqlite3
import requests
import time

from dotenv import load_dotenv, find_dotenv
from FinMind.data import DataLoader

In [2]:
# python 取得時間範圍內日期列表
# 來源：https://www.cnblogs.com/xiao-xue-di/p/11900649.html

def date_range(beginDate, endDate):
    dates = []
    dt = datetime.datetime.strptime(beginDate, "%Y-%m-%d")
    date = beginDate[:]
    while date <= endDate:
        dates.append(date)
        dt = dt + datetime.timedelta(1)
        date = dt.strftime("%Y-%m-%d")
    return dates

In [3]:
# 取得範圍日期列表(2019年12月30日至2026年2月18日)
price_date_list = date_range('2019-12-30', '2026-02-18')

In [4]:
# 設定FinMind API
load_dotenv(find_dotenv())
token = os.environ.get('FINMIND_TOKEN')
api = DataLoader()
api.login_by_token(api_token=token)

In [5]:
#API 使用次數
def api_usage() :
    url = "https://api.web.finmindtrade.com/v2/user_info"
    payload = {
        "token": token,
    }
    resp = requests.get(url, params=payload)
    return resp.json()["user_count"],resp.json()["api_request_limit"]

In [6]:
print(api_usage())

(666, 1600)


In [7]:
# 連線資料庫
conn = sqlite3.connect('data/stock.db')
cursor = conn.cursor()

In [8]:
# 啟用外鍵支持
conn.execute('PRAGMA foreign_keys = ON;')

res = conn.execute('PRAGMA foreign_keys;')
res.fetchone()

(1,)

In [9]:
# 設定資料庫結構
cursor.execute('''
CREATE TABLE IF NOT EXISTS StockInfo
(
    StockID TEXT NOT NULL PRIMARY KEY ,
    StockName TEXT,
    IndustryCategory TEXT,
    Type TEXT
);
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS DailyPrice
(
    SerialNo INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    StockID TEXT NOT NULL,
    Date TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume INTEGER,
    Value INTEGER,
    
    FOREIGN KEY (StockID) REFERENCES StockInfo (StockID)
);
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS WeeklyPrice
(
    SerialNo INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    StockID TEXT NOT NULL,
    Date TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume INTEGER,
    Value INTEGER,
    
    FOREIGN KEY (StockID) REFERENCES StockInfo (StockID)
);
''')
cursor.execute('''
CREATE TABLE IF NOT EXISTS MonthlyPrice
(
    SerialNo INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    StockID TEXT NOT NULL,
    Date TEXT,
    Open REAL,
    High REAL,
    Low REAL,
    Close REAL,
    Volume INTEGER,
    Value INTEGER,
    
    FOREIGN KEY (StockID) REFERENCES StockInfo (StockID)
);
''')
conn.commit()

In [10]:
# 台股總覽 TaiwanStockInfo #
df = api.taiwan_stock_info()
# display(df)
df_stock_info = df.drop(columns=['date'])
df_stock_info = df_stock_info.rename(columns={'stock_id':'StockID','stock_name':'StockName','industry_category':'IndustryCategory','type':'Type'})
df_stock_info.drop_duplicates(subset=['StockID'], keep='first', inplace=True)
display(df_stock_info)
for idx in range(df_stock_info.shape[0]) :
    info = df_stock_info.iloc[idx]
    conn.execute("INSERT INTO StockInfo(StockID,StockName,IndustryCategory,Type) VALUES(?,?,?,?)", (info['StockID'],info['StockName'],info['IndustryCategory'],info['Type']))

2026-02-18 16:36:19.258 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockInfo, data_id: 


,IndustryCategory,StockID,StockName,Type
0,電腦及週邊設備業,5450,寶聯通,tpex
1,文化創意業,3687,歐買尬,tpex
2,光電業,3629,地心引力,tpex
3,電子零組件業,5481,新華,tpex
4,其他電子類,6238,勝麗,tpex
...,...,...,...,...
4061,Index,BiotechnologyMedicalCare,生技醫療類指數,twse
4062,Index,Automobile,汽車類指數,twse
4063,大盤,TPEx,櫃買指數,tpex
4064,Index,Food,食品類指數,twse


In [11]:
for price_date in price_date_list :
    # API 使用次數 #
    user_count,api_request_limit = api_usage()
    if user_count > (api_request_limit - 100) :
        print('抓取資料速度過快（user_count＝ {} ，api_request_limit ＝ {}），等三十分鐘後再行抓取'.format(user_count,api_request_limit))
        time.sleep(30*60)
        
    # 股價日成交資訊 TaiwanStockPrice：一次拿特定日期，所有資料(只限 backer、sponsor 使用) #
    while True :
        try :
            df = api.taiwan_stock_daily(start_date=price_date,)
        except Exception as e:
            print('日K：日期{}發生錯誤{}，重試'.format(price_date,e))
            continue
        break
    if df.empty is not True :
        print('日K：{}'.format(price_date))
        df_daily_price = df.drop(columns=['spread','Trading_turnover'])
        df_daily_price = df_daily_price.rename(columns={'date':'Date','stock_id':'StockID','Trading_Volume':'Volume','Trading_money':'Value','open':'Open','max':'High','min':'Low','close':'Close'})
        # 保存格式：日期、股票代碼、開盤價、最高價、最低價、收盤價、成交量與成交值
        df_daily_price = df_daily_price[['Date', 'StockID', 'Open', 'High', 'Low', 'Close', 'Volume', 'Value']]
        # 排除掉非TaiwanStockInfo內的股票
        df_daily_price = df_daily_price[df_daily_price['StockID'].isin(df_stock_info['StockID'].to_list())]
        df_daily_price.to_sql('DailyPrice', conn, if_exists='append', index=False)
    else :    
        time.sleep(1)
    
    # 台股週 K 資料表 TaiwanStockWeekPrice (只限 backer、sponsor 會員使用) ： 一次拿特定日期，所有資料(只限 backer、sponsor 使用) #
    url = "https://api.finmindtrade.com/api/v4/data"
    parameter = {
        "dataset": "TaiwanStockWeekPrice",
        "start_date": price_date,
        "token": token,
    }
    while True :
        resp = requests.get(url, params=parameter)
        if resp.status_code == 200 :
            break
        else :
            print('週K：日期{}發生錯誤，回應狀態碼 ＝ {}'.format(price_date,resp.status_code))
            if resp.status_code == 402 :
                print('週K：API 用量超出上限，等十分鐘後重試')
                time.sleep(10*60)
    data = resp.json()
    df_weekly_price = pd.DataFrame(data["data"])
    if df_weekly_price.empty is not True :
        print('週K：{}'.format(price_date))
        df_weekly_price = df_weekly_price.drop(columns=['yweek','spread','trading_turnover'])
        df_weekly_price = df_weekly_price.rename(columns={'date':'Date','stock_id':'StockID','trading_volume':'Volume','trading_money':'Value','open':'Open','max':'High','min':'Low','close':'Close'})
        # 保存格式：日期、股票代碼、開盤價、最高價、最低價、收盤價、成交量與成交值
        df_weekly_price = df_weekly_price[['Date', 'StockID', 'Open', 'High', 'Low', 'Close', 'Volume', 'Value']]
        # 排除掉非TaiwanStockInfo內的股票
        df_weekly_price = df_weekly_price[df_weekly_price['StockID'].isin(df_stock_info['StockID'].to_list())]
        df_weekly_price.to_sql('WeeklyPrice', conn, if_exists='append', index=False)
    else :    
        time.sleep(1)
    
    # 台股月 K 資料表 TaiwanStockMonthPrice (只限 backer、sponsor 會員使用) ： 一次拿特定日期，所有資料(只限 backer、sponsor 使用) #
    url = "https://api.finmindtrade.com/api/v4/data"
    parameter = {
        "dataset": "TaiwanStockMonthPrice",
        "start_date": price_date,
        "token": token, 
    }
    while True :
        resp = requests.get(url, params=parameter)
        if resp.status_code == 200 :
            break
        else :
            print('月K：日期{}發生錯誤，回應狀態碼 ＝ {}'.format(price_date,resp.status_code))
            if resp.status_code == 402 :
                print('月K：API 用量超出上限，等十分鐘後重試')
                time.sleep(10*60)
    data = resp.json()
    df_monthly_price = pd.DataFrame(data["data"])
    if df_monthly_price.empty is not True :
        print('月K：{}'.format(price_date))
        df_monthly_price = df_monthly_price.drop(columns=['ymonth','spread','trading_turnover'])
        df_monthly_price = df_monthly_price.rename(columns={'date':'Date','stock_id':'StockID','trading_volume':'Volume','trading_money':'Value','open':'Open','max':'High','min':'Low','close':'Close'})
        # 保存格式：日期、股票代碼、開盤價、最高價、最低價、收盤價、成交量與成交值
        df_monthly_price = df_monthly_price[['Date', 'StockID', 'Open', 'High', 'Low', 'Close', 'Volume', 'Value']]
        # 排除掉非TaiwanStockInfo內的股票
        df_monthly_price = df_monthly_price[df_monthly_price['StockID'].isin(df_stock_info['StockID'].to_list())]
        df_monthly_price.to_sql('MonthlyPrice', conn, if_exists='append', index=False)
    else :    
        time.sleep(1)

2026-02-18 16:36:19.826 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


日K：2019-12-30
週K：2019-12-30


2026-02-18 16:36:23.447 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


日K：2019-12-31


2026-02-18 16:36:26.849 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 16:36:30.103 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


...

2026-02-18 21:28:17.585 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


日K：2026-02-10


2026-02-18 21:28:21.732 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


日K：2026-02-11


2026-02-18 21:28:25.859 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:29.435 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:33.013 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:36.596 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:40.182 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:43.745 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 
2026-02-18 21:28:47.375 | INFO     | FinMind.data.finmind_api:get_data:171 - download TaiwanStockPrice, data_id: 


In [12]:
# 修改並關閉資料庫
conn.commit()
conn.close()